In [1]:
import datetime
import gc
import joblib
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import os
import scipy.stats
import seaborn as sns
import sys
sys.path.append("../../")
import time
import warnings
warnings.simplefilter("ignore")
from collections import defaultdict
from itertools import repeat, combinations
from pandarallel import pandarallel
from sklearn.preprocessing import StandardScaler, LabelEncoder
from tqdm import tqdm

In [2]:
from utils.common import (
    sigmoid, pad_column_name
)
from utils.constants import *
from utils.eval_helpers import (
    plot_roc_curves, plot_feature_importance, 
    amex_metric, get_final_metric_df, amex_metric_np, lgb_amex_metric
)
from utils.eda_helpers import (
    plot_missing_proportion_barchart, plot_heatmap, plot_target_check,
    get_cols, insert_row_number, plot_train_test_distribution
)
from utils.extraction_helpers import read_file
from utils.feature_group import CATEGORY_COLUMNS
from utils.preprocess_helpers import clip_col

In [3]:
from feature_engineering_helpers import feature_gen_pipeline, clip_all, round_all, convert_all

In [4]:
%load_ext autoreload
%autoreload

In [5]:
pandarallel.initialize(nb_workers=16, progress_bar=False, use_memory_fs=False)

INFO: Pandarallel will run on 16 workers.
INFO: Pandarallel will use standard multiprocessing data transfer (pipe) to transfer data between the main process and workers.


In [6]:
START = time.time()

### Read Data

In [7]:
labels = read_file(f"../{RAW_DATA_PATH}/train_labels.csv")

Shape of data: (458913, 2)


In [8]:
%%time
train = read_file(f"../{INTERIM_DATA_PATH}/v7/train_parquet/train_all_variables.parquet")
test = read_file(f"../{INTERIM_DATA_PATH}/v7/test_parquet/test_all_variables.parquet")

Shape of data: (5531451, 158)
Shape of data: (11363762, 157)
CPU times: user 14.7 s, sys: 12.8 s, total: 27.5 s
Wall time: 16 s


In [9]:
df_list = [train, test]

## Impute

### D_10x

In [10]:
for df in tqdm(df_list):
    df["D_102"] = df.groupby("customer_ID")["D_102"].apply(lambda x: x.bfill().ffill())
    df["D_102"] = df["D_102"].fillna(1.06)
    df["D_102"] = np.where(df["D_102"] > 1, 1, df["D_102"])
    df["D_104"] = np.where(df["D_104"] > 1, 1, df["D_104"])
    df["D_104"] = df["D_104"].fillna(1.5)
    df["D_105"] = df["D_105"].fillna(-1)
    df["D_106"] = df["D_106"].fillna(50)
    df["D_107"] = df["D_107"].fillna(4)
    df["D_109"] = df["D_109"].fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [01:33<00:00, 46.56s/it]


### D_110 & D_111

In [11]:
for df in tqdm(df_list):
    df["D_110"] = df["D_110"].fillna(-1)
    df["D_111"] = df["D_111"].fillna(-1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.12s/it]


### D_112, D_115, D_118, D_119

In [12]:
for df in tqdm(df_list):
    df["D_112"] = df["D_112"].fillna(1)
    df["D_115"] = df["D_115"].fillna(-1)
    df["D_118"] = df["D_118"].fillna(0)
    df["D_119"] = df["D_119"].fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.84s/it]


### D_122 - D_130

In [13]:
for df in tqdm(df_list):
    df["D_122"] = df["D_122"].fillna(2)
    df["D_123"] = df["D_123"].fillna(1)
    df["D_124"] = df["D_124"].fillna(31)
    df["D_128"] = df["D_128"].fillna(0)
    df["D_129"] = df["D_129"].fillna(0)
    df["D_130"] = df["D_130"].fillna(1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.14s/it]


### D_131 - D_136

In [14]:
for df in tqdm(df_list):
    df["D_131"] = df["D_131"].fillna(1.25)
    df["D_132"] = df["D_132"].fillna(1.25)
    df.drop(columns="D_133", errors="ignore", inplace=True)
    df["D_134"] = df["D_134"].fillna(-1)
    df["D_136"] = df["D_136"].fillna(-2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:27<00:00, 14.00s/it]


### D_141 - D_145, D_183

In [15]:
for df in tqdm(df_list):
    df["D_141"] = df["D_141"].fillna(1)
    df["D_142"] = df["D_142"].fillna(-1)
    df["D_144"] = df["D_144"].fillna(-1)
    df["D_145"] = df["D_145"].fillna(2)
    df["D_183"] = df["D_183"].fillna(-2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.70s/it]


### D_39 - D_49

In [16]:
for df in tqdm(df_list):
    df["D_39"] = df["D_39"].fillna(0)
    df["D_41"] = df["D_41"].fillna(-1)
    df["D_42"] = df["D_42"].fillna(-0.01)
    df["D_43"] = df["D_43"].fillna(-0.01)
    df["D_44"] = df["D_44"].fillna(1)
    df["D_46"] = df["D_46"].fillna(0.4)
    df["D_48"] = df["D_48"].fillna(-1)
    df["D_49"] = df["D_49"].fillna(5)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:09<00:00,  4.93s/it]


### D_50 - D_58

In [17]:
for df in tqdm(df_list):
    df["D_50"] = df["D_50"].fillna(0)
    df["D_52"] = df["D_52"].fillna(0.09)
    df["D_53"] = df["D_53"].fillna(-1)
    df["D_54"] = df["D_54"].fillna(0.5)
    df["D_55"] = df["D_55"].fillna(-0.01)
    df["D_56"] = df["D_56"].fillna(0)
    df["D_58"] = df["D_58"].fillna(-1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:06<00:00,  3.34s/it]


### D_61, D_62, D_65

In [18]:
for df in tqdm(df_list):
    df["D_61"] = df["D_61"].fillna(-0.01)
    df["D_62"] = df["D_62"].fillna(0.19)
    df["D_65"] = df["D_65"].fillna(-2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.25it/s]


### D_70 - D_79

In [19]:
for df in tqdm(df_list):
    df["D_70"] = df["D_70"].fillna(1)
    df["D_71"] = df["D_71"].fillna(0)
    df["D_72"] = df["D_72"].fillna(0.5)
    df["D_74"] = df["D_74"].fillna(0)
    df["D_76"] = df["D_76"].fillna(-0.01)
    df["D_77"] = df["D_77"].fillna(0.05)
    df["D_78"] = df["D_78"].fillna(0)
    df["D_79"] = df["D_79"].fillna(0.5)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:10<00:00,  5.50s/it]


### D_8x & D_9x

In [20]:
for df in tqdm(df_list):
    df["D_80"] = df["D_80"].fillna(-1)
    df["D_81"] = df["D_81"].fillna(0.5)
    df["D_813"] = df["D_813"].fillna(0.5)
    df["D_83"] = df["D_83"].fillna(1)
    df["D_87"] = df["D_87"].fillna(0)
    df["D_87"] = df["D_87"].fillna(0)
    df["D_91"] = df["D_91"].fillna(-1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.18s/it]


### B_13 - B_19

In [21]:
for df in tqdm(df_list):
    df["B_13"] = df["B_13"].fillna(-1)
    df["B_15"] = df["B_15"].fillna(-0.1)
    df["B_16"] = df["B_16"].fillna(4)
    df["B_17"] = df["B_17"].fillna(1)
    df["B_19"] = df["B_19"].fillna(1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.31s/it]


### B_20 - B_26

In [22]:
for df in tqdm(df_list):
    df["B_20"] = df["B_20"].fillna(1)
    df["B_21"] = df["B_21"].fillna(-2)
    df["B_22"] = df["B_22"].fillna(0.5)
    df["B_24"] = df["B_24"].fillna(-2)
    df["B_25"] = df["B_25"].fillna(-0.1)
    df["B_26"] = df["B_26"].fillna(0)
    df["B_39"] = df["B_39"].fillna(2)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:04<00:00,  2.41s/it]


### B_2 - B_9

In [23]:
for df in tqdm(df_list):
    df["B_2"] = df["B_2"].fillna(2)
    df["B_3"] = df["B_3"].fillna(0.07)
    df["B_5"] = df["B_5"].fillna(0.03)
    df["B_6"] = df["B_6"].fillna(0.05)
    df["B_9"] = df["B_9"].fillna(0)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:02<00:00,  1.15s/it]


### P

In [24]:
for df in tqdm(df_list):
    df["P_2"] = df["P_2"].fillna(0.6)
    df["P_3"] = df["P_3"].fillna(0.4)
    df["P_3a"] = df["P_3a"].fillna(0.21)
    df["P_4"] = df["P_4"].fillna(0.07)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:01<00:00,  1.28it/s]


### R

In [25]:
for df in tqdm(df_list):
    df["R_10"] = df["R_10"].fillna(-2)
    df["R_11"] = df["R_11"].fillna(-1)
    df["R_12"] = df["R_12"].fillna(0.01)
    df["R_16"] = df["R_16"].fillna(-1)
    df["R_20"] = df["R_20"].fillna(-2)
    df["R_26"] = df["R_26"].fillna(-1)
    df["R_27"] = df["R_27"].fillna(2)
    df["R_5"] = df["R_5"].fillna(-2)
    df["R_6"] = df["R_6"].fillna(-2)
    df["R_8"] = df["R_8"].fillna(-1)
    df["R_9"] = df["R_9"].fillna(-1)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:05<00:00,  2.63s/it]


### S

In [26]:
for df in tqdm(df_list):
    df["S_3"] = df["S_3"].fillna(0.11)
    df["S_7"] = df["S_7"].fillna(0.08)
    df["S_8"] = df["S_8"].fillna(0.36)
    df["S_12"] = df["S_12"].fillna(-1)
    df["S_13"] = df["S_13"].fillna(1)
    df["S_15"] = df["S_15"].fillna(0.4)
    df["S_16"] = df["S_16"].fillna(-1)
    df["S_16"] = df["S_16"].fillna(-1)
    df["S_23"] = df["S_23"].fillna(10)
    df["S_25"] = df["S_25"].fillna(10)
    df["S_26"] = df["S_26"].fillna(0.05)
    df["S_27"] = df["S_27"].fillna(0.4).replace(0, 5)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 2/2 [00:03<00:00,  1.98s/it]


In [27]:
# plot_target_check(train, "S_27", q=50, use_raw_bin=True, nunique_thr=2000, strfy_x=True, without_drop_tail=False)

In [28]:
m = plot_missing_proportion_barchart(train, show_plot=False)
# m.loc[m["column"].str.contains("S_")].sort_values(by="column")
m.sort_values(by="column")

,column,missing_proportion


In [32]:
train.to_pickle("imputed_train.pkl")

In [33]:
test.to_pickle("imputed_test.pkl")

### Transform Train Set

In [ ]:
%%time
train_agg, keep_column = feature_gen_pipeline(train)

In [ ]:
train_agg["target"] = labels["target"].values
print(f"Size: {sys.getsizeof(train_agg) / 1e9} GB, Shape: {train_agg.shape}")

In [ ]:
train_agg.to_parquet("./train_agg.parquet")

In [ ]:
train_agg = convert_all(train_agg)

In [ ]:
train_agg = clip_all(train_agg)

In [ ]:
train_agg = round_all(train_agg)

In [ ]:
print(f"Size: {sys.getsizeof(train_agg) / 1e9} GB, Shape: {train_agg.shape}")

In [ ]:
train_agg.shape

In [ ]:
train_agg.to_parquet(f"./train_agg.parquet")

In [ ]:
del train_agg, train

### Transform Test

In [ ]:
test = pd.concat([test1, test2], ignore_index=True)

In [ ]:
del test1, test2

In [ ]:
%%time
test_agg, keep_column = feature_gen_pipeline(test)

In [ ]:
print(f"Size: {sys.getsizeof(test_agg) / 1e9} GB, Shape: {test_agg.shape}")

In [ ]:
test_agg.to_parquet("./test_agg.parquet")

In [ ]:
test_agg = convert_all(test_agg)

In [ ]:
test_agg = clip_all(test_agg)

In [ ]:
test_agg = round_all(test_agg)

In [ ]:
print(f"Size: {sys.getsizeof(test_agg) / 1e9} GB, Shape: {test_agg.shape}")

In [ ]:
test_agg.shape

In [ ]:
test_agg.to_parquet(f"./test_agg.parquet")